# 2) Geospatial Analysis with GeoPandas

### Task 1: Static Maps — Hospital Count by District

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from shapely.geometry import Point

In [ ]:
districts = gpd.read_file(r'C:/Users/Carla HL/Desktop/DIPLOMATURA Q-LAB/PYTHON INTERMEDIO/TAREA 2/dist_shape_file/DISTRITOS.shp')
districts.head(5)

In [ ]:
#Check unique values
districts['IDDIST'].is_unique

In [ ]:
# Select only relevant columns
districts = districts[['IDDIST', 'geometry']]
districts = districts.rename({'IDDIST':'UBIGEO'}, axis =1 )

In [ ]:
districts['UBIGEO'] = districts['UBIGEO'].astype(str).astype(int)
districts.head(5)

In [ ]:
# Ensure the dataset is in WGS-84 (EPSG:4326)
districts = districts.to_crs(epsg=4326)
districts.crs

##### Data: Hospitales

In [ ]:
hospitals=pd.read_csv(r'C:/Users/Carla HL/Desktop/DIPLOMATURA Q-LAB/PYTHON INTERMEDIO/TAREA 2/IPRESS.csv', encoding='latin-1')
hospitals.head(5)

In [ ]:
# Filtros: hospitales EN FUNCIONAMIENTO con coordenadas válidas
operational_hospitals = hospitals[hospitals['Condición'] == 'EN FUNCIONAMIENTO'].copy()
operational_hospitals = operational_hospitals.dropna(subset=['NORTE', 'ESTE'])

# Se obtiene el número de hospitales por distrito
operational_hospitals["n_hospitals_distr"]=operational_hospitals.groupby('UBIGEO')['UBIGEO'].transform("count")
operational_hospitals.head(5)

In [ ]:
# Merge left: para mantener todos los distritos
hospitals_distr = pd.merge(districts, operational_hospitals, how="left", on="UBIGEO")

# Se asigna 0 a los distritos en los que no hay registro de hospitales en funcionamiento 
hospitals_distr["n_hospitals_distr"]=hospitals_distr["n_hospitals_distr"].fillna(0).astype(int)

##### MAP 1: Total public hospitals per district

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

hospitals_distr.plot(column='n_hospitals_distr', 
                          cmap='Blues', 
                          legend=True,
                          ax=ax,
                          edgecolor='black',
                          linewidth=0.1)

ax.set_title('Total public hospitals per district', fontsize=14)
ax.set_xlabel('Longitud', fontsize=10)
ax.set_ylabel('Latitud', fontsize=10)

# Add colorbar label
cbar = ax.get_figure().get_axes()[-1]
cbar.set_ylabel('Number of Hospitals', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

##### MAP 2: Highlight districts with zero hospitals

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Plotear todos los distritos en gris claro como referencia
hospitals_distr.plot(
    color="lightgrey",
    edgecolor="black",
    linewidth=0.1,
    ax=ax
)

# Resaltar en rojo los distritos con 0 hospitales
hospitals_distr[hospitals_distr["n_hospitals_distr"] == 0].plot(
    color="red",
    edgecolor="black",
    linewidth=0.3,
    ax=ax,
)

ax.set_title("Districts with zero hospitals", fontsize=14)
ax.set_xlabel("Longitude", fontsize=10)
ax.set_ylabel("Latitude", fontsize=10)

plt.tight_layout()
plt.show()

##### Map 3: Top 10 districts with the highest number of hospitals

In [ ]:
# Seleccionar top 10 distritos
by_distr = (
    hospitals_distr[["UBIGEO", "geometry", "n_hospitals_distr"]]
    .drop_duplicates(subset="UBIGEO")
)
top10 = by_distr.nlargest(10, "n_hospitals_distr")

# Graficar todos los distritos como fondo en gris claro
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
hospitals_distr.plot(
    color="lightgrey",
    edgecolor="black",
    linewidth=0.1,
    ax=ax
)

# Graficar top 10 con una escala de colores
top10.plot(
    column="n_hospitals_distr",
    cmap="Blues",    
    legend=True,
    ax=ax,
    edgecolor="black",
    linewidth=0.4
)

ax.set_title("Top 10 districts with the highest number of hospitals", fontsize=14)
ax.set_xlabel("Longitude", fontsize=10)
ax.set_ylabel("Latitude", fontsize=10)

# Etiqueta de la barra de color
cbar = ax.get_figure().get_axes()[-1]
cbar.set_ylabel("Number of Hospitals", rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

### Task 2: Department-level Analysis

In [ ]:
operational_hospitals_dept = (
    operational_hospitals
    .groupby("Departamento")["n_hospitals_distr"]
    .sum()
    .reset_index()
    .rename(columns={"n_hospitals_distr": "n_hospitals_dept"})
)
operational_hospitals_dept
operational_hospitals_dept.to_csv("summary_table.csv", index=False)

In [ ]:
# Department with the highest number of hospitals
max_row = operational_hospitals_dept.loc[operational_hospitals_dept["n_hospitals_dept"].idxmax()]

# Department with the lowest number of hospitals
min_row = operational_hospitals_dept.loc[operational_hospitals_dept["n_hospitals_dept"].idxmin()]

print("Department with the most hospitals:")
print(max_row["Departamento"], "-", max_row["n_hospitals_dept"])

print("\nDepartment with the fewest hospitals:")
print(min_row["Departamento"], "-", min_row["n_hospitals_dept"])

##### Summary table (sorted from highest to lowest)

In [ ]:
summary_table = (
    operational_hospitals_dept[["Departamento", "n_hospitals_dept"]]
    .sort_values(by="n_hospitals_dept", ascending=False)
    .reset_index(drop=True)
)

print(summary_table)

##### Bar chart

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.barplot(
    data=summary_table,
    x="Departamento",
    y="n_hospitals_dept",
    color="steelblue"
)

plt.xticks(rotation=90)
plt.xlabel("Department")
plt.ylabel("Number of Hospitals")
plt.title("Number of Hospitals by Department")

# Añadir etiquetas encima de cada barra
for p in ax.patches:
    ax.annotate(
        format(p.get_height(), '.0f'),     
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='bottom',        
        fontsize=9, color='black', xytext=(0,1), textcoords='offset points'
    )

plt.tight_layout()
plt.savefig("hospitals_dept_bars.png",dpi=300, bbox_inches="tight")
plt.show()

##### Department-level choropleth map

In [ ]:
# Aggregate counts to department & dissolve geometry
operational_hospitals_dept = (
    hospitals_distr
      .dissolve(
          by="Departamento",
          aggfunc={"n_hospitals_distr": "sum"}
      )
      .rename(columns={"n_hospitals_distr": "n_hospitals_dept"})
      .reset_index()
)

# Plot choropleth
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
operational_hospitals_dept.plot(
    column="n_hospitals_dept",
    cmap="Blues",
    legend=True,
    edgecolor="black",
    linewidth=0.3,
    ax=ax
)

ax.set_title("Hospitals by Department", fontsize=14)
ax.set_xlabel("Longitude"); ax.set_ylabel("Latitude")

# Label for the colorbar
cbar = ax.get_figure().get_axes()[-1]
cbar.set_ylabel("Number of Hospitals", rotation=270, labelpad=20)

plt.tight_layout()
plt.savefig("hospitals_dept_map.png",dpi=300, bbox_inches="tight")
plt.show()

# 3) Interactive Mapping with Folium

### Task 1: National Choropleth (District Level)

In [ ]:
# geodata en formato geojson
geo_data=districts.to_json()

In [ ]:
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster

In [ ]:
# Center the map
m = folium.Map(location=[-9.2, -75.0], zoom_start=5, tiles="cartodbpositron")

# Choropleth:
ch = folium.Choropleth(
    geo_data=geo_data,
    name="Hospitals per district",
    data=by_distr,
    columns=["UBIGEO", "n_hospitals_distr"],
    key_on="feature.properties.UBIGEO",
    fill_color="Blues",
    fill_opacity=0.8,
    line_opacity=0.4,
    nan_fill_color="lightgray",
    legend_name="Hospitals per district"
).add_to(m)

# Cluster markers
# Coordinates in a list
hospitales = list(zip(operational_hospitals['ESTE'], operational_hospitals['NORTE']))

# Cluster Map
MarkerCluster(hospitales, name = 'Cluster').add_to(m)

folium.LayerControl().add_to(m)
m